In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import SnowballStemmer
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.cluster import KMeans

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
print(device)

нам нужно идти по трейн дф и загружать картинки по именам из колонки, пофиг на повторения. Удаляем дубликаты из трейн дф и картиночного массива

можно попробовать к нейборс с 50 соседями

In [ ]:
# Сбор данных

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')
train_df = pd.DataFrame(train)
copy_train = train_df.copy()
train_df.head()

In [ ]:
train_df['label_group'].shape

In [ ]:
test = pd.read_csv('../input/shopee-product-matching/test.csv')
test_df = pd.DataFrame(test)
test_df.head()

In [ ]:
#32412
pics_train = 34250
pics_test = 3
size = 28
train_images = np.empty(shape=(pics_train, size ** 2))
test_images = np.empty(shape=(pics_test, size ** 2))
print(train_images.shape, test_images.shape)

In [ ]:
names = train_df['image']
dirname = '../input/shopee-product-matching/train_images/'
print(len(names))
for i in range(len(names)):
    img = cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(dirname, names[i])), cv2.COLOR_BGR2GRAY) / 255., (size, size)).flatten()
    print(os.path.join(dirname, names[i]), img.shape)
    train_images[i] = img

In [ ]:
'''train_images = np.unique(train_images)
train_df.drop_duplicates(subset=['image'], ignore_index=True, inplace=True)'''

In [ ]:
print(train_df.shape, train_images.shape)
print(test_df.shape, test_images.shape)

In [ ]:
for dirname, _, filenames in os.walk('../input/shopee-product-matching/test_images/'):
    for filename in filenames:
        img = cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(dirname, filename)), cv2.COLOR_BGR2GRAY) / 255., (size, size)).flatten()
        print(os.path.join(dirname, filename), img.shape)
        test_images[filename.index(filename)] = img

# Классификация картинок

In [ ]:
train_data = torch.from_numpy(train_images).float()
train_data = train_data.to(device)
unique = train_df['label_group'].unique()
ids = np.array([i for i in range(len(unique))])
id_dict = dict(zip(unique, ids))
id_labels = np.array([id_dict[i] for i in train_df['label_group'].values])
id_labels = torch.from_numpy(id_labels).long()
id_labels = id_labels.to(device)
print(max(id_labels))

In [ ]:
test_data = torch.from_numpy(test_images).float()
test_data = test_data.to(device)

In [ ]:
num_features = train_data.shape[1]
hidden_units = 192
num_classes = train_df['label_group'].nunique()
model = nn.Sequential(
nn.Linear(num_features, hidden_units, bias=True),
nn.LeakyReLU(),
nn.Linear(hidden_units, hidden_units, bias=True),
nn.LeakyReLU(),
nn.Linear(hidden_units, num_classes)
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.to(device)

In [ ]:
cost = []
criterion = nn.CrossEntropyLoss()

In [ ]:
def fit(X, Y, epochs):
    for i in range(epochs):
        preds = model(X)
        loss = criterion(preds, Y)
        cost.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'loss on iter {i} is {cost[-1]}')

In [ ]:
def predict(X):
    preds = nn.Softmax(model(X))
    return preds

In [ ]:
fit(train_data, id_labels, 430)

In [ ]:
plt.plot(cost)

In [ ]:
predict(test_data)